In [97]:
# Part 2

In [1]:
# OpenAI API through langchain to create a stateful, context aware and reasining llm powered application

In [ ]:
# Model I/O module - includes a model, prompt, example selector and output parser + prompt templates
# Model inputs include prmompt templates abstraction to prompts making them resuasble
# Examples : I have recently adopted a {pet}. Could you suggest some {pet} names? Then provide many pet examples to train called 
# Few shot prompting
# Model outputs - Models return o/p as assistant-role message which are unsuitable for a python script that expects a pandas dataframe object
# or langiage models that require xml input such as those by Anthropic. This is where output parsers are useful: String Output parser
# DateTime output parser and Comma-separated List Output parser
# Retrieval module - Retrievers are components we can add to our chatbot to make them context aware., we can strategically feed them external or proprietary
#information  so they can answer questions on data that they have not trained on.
# Components are Document Loader, Text splitter, Embedding Model, Vector store, Retriever
# We create a chatbot to?
# Agent tooling allows reasoning chatbots to be created which choses from a set of tools and orders the steps necessary for completing a task
# tools and toolkits. You can create a toolkit and integrate it to framework
# Lang Chain expression language is the specific protocol that links the three modules
# Templates are preimpplemented templates that have proved helpful for a specific task.
# LanSmith platform is responsible for products observability
# Langserve is for the product's deployment.


In [ ]:
# Create a helpful sarcastic chatbot in Lang Chain

In [9]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
pip install langchain_openai

In [11]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [17]:
chat = ChatOpenAI(model='gpt-4', seed=365, temperature=0, max_tokens='100') # Use a seed paramter to obtain similar results to other runs

In [19]:
response = chat.invoke('''I have recently adopted a dog. Can you suggest a dog name?''') 
# invoke is the chief method in Langchain. ''' allows to move me to move seamlessly moving text to new line when prompt becomes lengthy
# You can also us ' and " as part of the prompt without terminating the string. """ has the same effect

In [20]:
print(response.content)

Sure, how about "Max"?


In [ ]:
# Using System Role and Human Role ( user role in gpt)

In [23]:
message_h_dog = HumanMessage(content='''I have recently adopted a dog. Can you suggest a dog name?''')
message_s= SystemMessage(content='''You are Marv, a chatbot that reluctantly answers questions with sarcastic responses''')
message_ai_dog = AIMessage(''' Oh, absolutely. Because nothing screams "I\'m a responsible dog owner" like asking a chatbot to name your new best friend. How about "Bark Twain" or "Sir wag-a-lot''')

In [ ]:
# invoke method accepts a list of chat messages including Human and System message
response = chat.invoke([message_s,message_h])

In [39]:
response

AIMessage(content='Oh, absolutely. Because nothing screams "I\'m a responsible pet owner" like asking a chatbot to name your new best friend. How about "Bark Twain" or "Sir Wag-a-lot"? You know, something that really captures the essence of your deep bond.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 41, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-552cb0b8-43e2-4cf9-bcd0-100cf35b2fb0-0', usage_metadata={'input_tokens': 41, 'output_tokens': 58, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# AI message is the assistant role message in Openai's API - give examples called Few Shot prompting

In [27]:
message_ai = AIMessage(''' Oh, absolutely. Because nothing screams "I\'m a responsible pet owner" like asking a chatbot to name your new best friend. How about "Mew Twain" or "Sir purr-a-lot''')

In [29]:
message_h_cat = HumanMessage(content='''I have recently adopted a cat. Can you suggest a cat name?''')
message_ai_cat = AIMessage(''' Oh, absolutely. Because nothing screams "I\'m a responsible cat owner" like asking a chatbot to name your new best friend. How about "Catastrophe" or "Furry-face''')
message_h_fish = HumanMessage(content='''I have recently adopted a fish. Can you suggest a fish name? ''')

In [31]:
response2 = chat.invoke([message_h_dog, message_ai_dog, message_h_cat, message_ai_cat, message_h_fish])

In [32]:
response2

AIMessage(content='Of course! How about "Finley" or "Bubbles"?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 148, 'total_tokens': 163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c4cd5f96-4d4b-4077-9491-fe188b9649c8-0', usage_metadata={'input_tokens': 148, 'output_tokens': 15, 'total_tokens': 163, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [35]:
# 1 examples did not get the intended response but couple of examples improved it. This comes at a cost of using more tokens

In [ ]:
# create abstraction for our prompts and make them reusable using cta prompt templates

In [83]:
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [41]:
# String template
TEMPLATE=''' 
System:
{description}

Human:
I've recently adopted a {pet}
Could you suggest some {pet} names?
'''

In [47]:
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [49]:
prompt_template

PromptTemplate(input_variables=['description', 'pet'], input_types={}, partial_variables={}, template=" \nSystem:\n{description}\n\nHuman:\nI've recently adopted a {pet}\nCould you suggest some {pet} names?\n")

In [53]:
prompt_value = prompt_template.invoke({'description':'''The chatbot should reluctantly answer questions with sarcastic 
responses ''', 'pet':'dog'})

In [55]:
prompt_value


StringPromptValue(text=" \nSystem:\nThe chatbot should reluctantly answer questions with sarcastic \nresponses \n\nHuman:\nI've recently adopted a dog\nCould you suggest some dog names?\n")

In [59]:
TEMPLATE_S='{description}'

In [63]:
TEMPLATE_H=''' I have recently adopted a {pet}.
Could you suggest some {pet} names'''

In [61]:
message_template_s = SystemMessagePromptTemplate.from_template(TEMPLATE_S)

In [108]:
message_template_s

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['description'], input_types={}, partial_variables={}, template='{description}'), additional_kwargs={})

In [65]:
message_template_h = HumanMessagePromptTemplate.from_template(TEMPLATE_H)

In [67]:
message_template_h

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template=' I have recently adopted a {pet}.\nCould you suggest some {pet} names'), additional_kwargs={})

In [69]:
chat_template = ChatPromptTemplate.from_messages([message_template_s, message_template_h])

In [71]:
chat_template

ChatPromptTemplate(input_variables=['description', 'pet'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['description'], input_types={}, partial_variables={}, template='{description}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template=' I have recently adopted a {pet}.\nCould you suggest some {pet} names'), additional_kwargs={})])

In [73]:
chat_value = chat_template.invoke({ 'description': '''The chatbot should reluctantly answer questions with sarcastic responses ''', 
                                   'pet':'dog'})

In [75]:
chat_value

ChatPromptValue(messages=[SystemMessage(content='The chatbot should reluctantly answer questions with sarcastic responses ', additional_kwargs={}, response_metadata={}), HumanMessage(content=' I have recently adopted a dog.\nCould you suggest some dog names', additional_kwargs={}, response_metadata={})])

In [79]:
# invoke accepts instance of the chat prompt class
response3 = chat.invoke(chat_value)

In [80]:
response3

AIMessage(content='Oh, absolutely. Because I\'m just brimming with dog names. How about "Bark Twain" or "Sir Wag-a-lot"? Maybe "Fur-dinand" or "Bark Obama"? Or if you\'re feeling particularly creative, "Dog".', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 36, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d5d1b4fe-8a72-44cb-a75d-66fb641cd1cd-0', usage_metadata={'input_tokens': 36, 'output_tokens': 55, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# In summary ChatpromptTemplate accepts a dictiionary and returns a chatpromptvalue object. The chatOpenAI method accepts a chatpromptvalue and
# returns  AIMessage. Output of one invoke goes as input to the other - concept of LCEL

In [ ]:
# Most straightforward chain class LLM Chain

In [132]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [85]:
from langchain.chains.llm import LLMChain

In [87]:
chat_template

ChatPromptTemplate(input_variables=['description', 'pet'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['description'], input_types={}, partial_variables={}, template='{description}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template=' I have recently adopted a {pet}.\nCould you suggest some {pet} names'), additional_kwargs={})])

In [89]:
chain = LLMChain(llm=chat, prompt=chat_template)

/var/folders/55/z4lt05n13lggvkf4cntq7dc40000gr/T/ipykernel_42661/3887436020.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=chat_template)


In [91]:
response4 = chain.invoke({ 'description': 'The chatbot should reluctantly answer questions with sarcastic responses ','pet':'dog'})

In [93]:
response4

{'description': 'The chatbot should reluctantly answer questions with sarcastic responses ',
 'pet': 'dog',
 'text': 'Oh, absolutely, because I\'ve been waiting all day to help someone name their dog. How about "Bark Twain" or "Sir Waggington"? Or if you\'re into pop culture, "Bark Vader" or "Wooferine"? I mean, who wouldn\'t want their dog to have a name that\'s a pun?'}